In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [3]:
digits = pd.read_csv('../data/digit_recognizer_train.csv')

In [4]:
digits.shape

(42000, 785)

In [5]:
digits.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [6]:
X = digits.iloc[:, 1:]
y = digits.iloc[:, 0]

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.fit_transform(X_valid)

X_test = sc.transform(X_test)

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [15]:
model = Sequential()
model.add(Flatten())
model.add( Dense(128, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))   )
model.add( Dense(64, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l1(l=0.001))  )
model.add( Dense(10, activation = 'softmax'))

#### Model Check Point Call Back

In [16]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD())

checkpoint_callback = ModelCheckpoint("./models/1/digit_ckpt_keras_model.{epoch:02d}-{val_loss:.2f}.h5", save_best_only=True)

history = model.fit(X_train, y_train.values, epochs=10,
                    validation_data=(X_valid, y_valid.values),
                    callbacks=[checkpoint_callback])

model.save("digit_ckpt_keras_model.h5")

model = load_model("digit_ckpt_keras_model.h5") # rollback to best model
print(model.evaluate(X_test, y_test.values))

Epoch 1/10
840/840 [==============================] - 1s 1ms/step - loss: 5.7074 - val_loss: 4.3020
Epoch 2/10
840/840 [==============================] - 1s 1ms/step - loss: 4.0676 - val_loss: 3.4821
Epoch 3/10
840/840 [==============================] - 1s 1ms/step - loss: 3.2821 - val_loss: 2.7989
Epoch 4/10
840/840 [==============================] - 1s 1ms/step - loss: 2.6210 - val_loss: 2.2272
Epoch 5/10
840/840 [==============================] - 1s 1ms/step - loss: 2.0780 - val_loss: 1.7542
Epoch 6/10
840/840 [==============================] - 1s 1ms/step - loss: 1.6306 - val_loss: 1.3920
Epoch 7/10
840/840 [==============================] - 1s 1ms/step - loss: 1.2924 - val_loss: 1.1188
Epoch 8/10
840/840 [==============================] - 1s 1ms/step - loss: 1.0508 - val_loss: 0.9360
Epoch 9/10
840/840 [==============================] - 1s 1ms/step - loss: 0.8775 - val_loss: 0.8166
Epoch 10/10
263/263 [==============================] - 0s 661us/step - loss: 0.8095
0.80950349569320

In [17]:
from sklearn.metrics import accuracy_score

In [27]:
y_pred = model.predict(X_test)

In [28]:
y_pred.shape

(8400, 10)

In [32]:
y_pred[1]

array([1.6268167e-04, 9.8381084e-01, 2.3262992e-03, 2.0189798e-03,
       4.4764986e-04, 1.9343352e-03, 1.2929513e-03, 1.9880263e-03,
       5.3740940e-03, 6.4420037e-04], dtype=float32)

In [33]:
np.argmax(y_pred[1])

1

In [34]:
y_pred = np.argmax(y_pred, axis =1)
print(y_pred)

[8 1 9 ... 3 0 9]


In [36]:
accuracy_score(y_test, y_pred)

0.9441666666666667

In [37]:
model.predict(X_test)

array([[9.7506400e-04, 1.7963130e-02, 3.5624377e-02, ..., 8.3646580e-04,
        9.3254596e-01, 4.3755109e-04],
       [1.6268167e-04, 9.8381084e-01, 2.3262992e-03, ..., 1.9880263e-03,
        5.3740940e-03, 6.4420037e-04],
       [2.6364459e-04, 2.8586353e-04, 9.3295428e-05, ..., 3.0094042e-02,
        1.6881042e-03, 9.3501198e-01],
       ...,
       [7.9149926e-05, 3.9242712e-04, 4.8487663e-04, ..., 9.3709496e-05,
        9.7047072e-04, 2.1824983e-04],
       [9.8072219e-01, 2.0462276e-04, 2.6375107e-03, ..., 3.7204067e-03,
        2.4155817e-04, 5.0028990e-04],
       [3.6802128e-04, 3.1165319e-04, 1.3954145e-03, ..., 7.8226263e-03,
        5.0225728e-03, 9.5235503e-01]], dtype=float32)

#### Early Stopping & Model Check Point call back

In [38]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint("./models/digit_ckpt_keras_model.h5", save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD())

history = model.fit(X_train, y_train.values, epochs=100,
                    validation_data=(X_valid, y_valid.values),
                    callbacks=[checkpoint_callback, early_stopping_callback])

model = load_model("digit_ckpt_keras_model.h5") # rollback to best model
print(model.evaluate(X_test, y_test.values))

Epoch 1/100
840/840 [==============================] - 1s 1ms/step - loss: 0.7170 - val_loss: 0.7055
Epoch 2/100
840/840 [==============================] - 1s 1ms/step - loss: 0.6711 - val_loss: 0.6570
Epoch 3/100
840/840 [==============================] - 1s 1ms/step - loss: 0.6249 - val_loss: 0.6246
Epoch 4/100
840/840 [==============================] - 1s 1ms/step - loss: 0.5894 - val_loss: 0.5935
Epoch 5/100
840/840 [==============================] - 1s 1ms/step - loss: 0.5618 - val_loss: 0.5637
Epoch 6/100
840/840 [==============================] - 1s 1ms/step - loss: 0.5364 - val_loss: 0.5389
Epoch 7/100
840/840 [==============================] - 1s 1ms/step - loss: 0.5073 - val_loss: 0.5180
Epoch 8/100
840/840 [==============================] - 1s 1ms/step - loss: 0.4935 - val_loss: 0.5090
Epoch 9/100
840/840 [==============================] - 1s 1ms/step - loss: 0.4750 - val_loss: 0.4938
Epoch 10/100
840/840 [==============================] - 1s 1ms/step - loss: 0.4583 - val_lo

In [39]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis =1)
accuracy_score(y_test, y_pred)

0.9441666666666667

#### Custom Call Back - Print Debug every n-epoches

In [44]:
def conditalPrint(batch_num, logs):
    if(batch_num%128 == 0):
        print(batch_num , " :::: ", logs)

batch_print_callback = tf.keras.callbacks.LambdaCallback(
    on_batch_begin=lambda batch,logs: conditalPrint(batch, logs))

In [45]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD())

history = model.fit(X_train, y_train.values, epochs=10,
                    validation_data=(X_valid, y_valid.values),
                    callbacks=[batch_print_callback])

print(model.evaluate(X_test, y_test.values))

Epoch 1/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.4407 - val_loss: 0.4561
Epoch 2/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.4180 - val_loss: 0.4482
Epoch 3/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.4144 - val_loss: 0.4466
Epoch 4/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.4009 - val_loss: 0.4300
Epoch 5/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.3871 - val_loss: 0.4252
Epoch 6/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.3881 - val_loss: 0.4156
Epoch 7/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.3779 - val_loss: 0.4038
Epoch 8/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.3615 - val_loss: 0.3995
Epoch 9/10
0  ::::  {}
840/840 [==============================] - 1s 1ms/step - loss: 0.3594 - val_loss: